# Epistatic Interactions
This does a better job than the other notebooks, and is based on the code from [Phillips et al.](https://elifesciences.org/articles/71393#s4)

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import sys
import re
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MultiLabelBinarizer
import statsmodels.api as sm
import pandas as pd

mutations_9114 = ['S29F', 'N30S', 'N31S', 'S52I', 'S56T', 'T57A', 'A58N',
                  'S70T', 'I73K', 'F74S', 'S75T', 'N76S', 'N82AS', 'T83R', 'F91Y', 'S100BY']
mutations_6261 = ['P28T', 'R30S', 'T57A', 'K58N', 'P61Q',
                  'D73E', 'F74S', 'A75T', 'G76S', 'V78A', 'V100L']


## Read in data

In [18]:
data = pd.read_csv("~/rosetta-antibody-ddgs/raw_datasets/full_data.csv")

# Subsetting for phillips data
new_data = data[~data["Source"].str.contains("Phillips")]
data = data[data["Source"].str.contains("Phillips")]

mut_1 = data.loc[data["LD"] == 1]
mut_2 = data.loc[data["LD"] == 2]
mut_3 = data.loc[data["LD"] == 3]
mut_4 = data.loc[data["LD"] == 4]
mut_5 = data.loc[data["LD"] == 5]
mut_2 = mut_2.drop_duplicates(subset="ddG(kcal/mol)", keep=False).copy()
mut_3 = mut_3.drop_duplicates(subset="ddG(kcal/mol)", keep=False).copy()
mut_4 = mut_4.drop_duplicates(subset="ddG(kcal/mol)", keep=False).copy()
mut_5 = mut_5.drop_duplicates(subset="ddG(kcal/mol)", keep=False).copy()

dump_data = pd.concat([mut_1, mut_2, mut_3, mut_4, mut_5]
                      ).reset_index(drop=True)
print(dump_data.head())


   #PDB Mutations  ddG(kcal/mol)                Source  LD
0  3GBN    H:D73E      -0.123782  Phillips et al. 2021   1
1  3GBN    H:T57A       0.007413  Phillips et al. 2021   1
2  3GBN    H:V78A       0.011997  Phillips et al. 2021   1
3  3GBN    H:P61Q       0.012366  Phillips et al. 2021   1
4  3GBN    H:G76S       0.058492  Phillips et al. 2021   1


In [19]:
mlb9114 = MultiLabelBinarizer(classes=mutations_9114, sparse_output=True)
mlb6261 = MultiLabelBinarizer(classes=mutations_6261, sparse_output=True)

dump_data9114 = dump_data.loc[dump_data["#PDB"] == "4FQY"].copy()
dump_data6261 = dump_data.loc[dump_data["#PDB"] == "3GBN"].copy()

dump_data9114.loc[:, ("Mutations")] = dump_data9114.loc[:, ("Mutations")].apply(
    lambda x: re.split(";", re.sub(r"\w:(\w+)", r"\1", x))).copy()
dump_data6261.loc[:, ("Mutations")] = dump_data6261.loc[:, ("Mutations")].apply(
    lambda x: re.split(";", re.sub(r"\w:(\w+)", r"\1", x))).copy()

dump_data9114 = dump_data9114.join(pd.DataFrame.sparse.from_spmatrix(
    mlb9114.fit_transform(dump_data9114.pop('Mutations')),
    index=dump_data9114.index,
    columns=mlb9114.classes_))
dump_data9114.drop("Source", axis=1, inplace=True)
dump_data6261 = dump_data6261.join(pd.DataFrame.sparse.from_spmatrix(
    mlb6261.fit_transform(dump_data6261.pop('Mutations')),
    index=dump_data6261.index,
    columns=mlb6261.classes_))
dump_data6261.drop("Source", axis=1, inplace=True)
print(dump_data9114.head())
print(dump_data6261.head())


    #PDB  ddG(kcal/mol)  LD  S29F  N30S  N31S  S52I  S56T  T57A  A58N  S70T  \
11  4FQY      -0.587729   1     0     0     0     0     0     0     1     0   
12  4FQY      -0.159545   1     0     0     0     0     0     0     0     0   
13  4FQY       0.805839   1     0     0     0     0     0     0     0     0   
14  4FQY       1.163809   1     0     0     0     0     0     1     0     0   
15  4FQY       0.451638   1     1     0     0     0     0     0     0     0   

    I73K  F74S  S75T  N76S  N82AS  T83R  F91Y  S100BY  
11     0     0     0     0      0     0     0       0  
12     0     0     0     0      0     0     1       0  
13     0     0     0     1      0     0     0       0  
14     0     0     0     0      0     0     0       0  
15     0     0     0     0      0     0     0       0  
   #PDB  ddG(kcal/mol)  LD  P28T  R30S  T57A  K58N  P61Q  D73E  F74S  A75T  \
0  3GBN      -0.123782   1     0     0     0     0     0     1     0     0   
1  3GBN       0.007413   1     0 

In [23]:
affinities_6261 = dump_data6261[["ddG(kcal/mol)"]].values.flatten()
print(affinities_6261.shape)
genotypes_6261 = np.array(
    dump_data6261[[x for x in mutations_6261]].copy(), dtype=np.float64)
print(genotypes_6261.shape)

affinities_9114 = dump_data9114[["ddG(kcal/mol)"]].values.flatten()
print(affinities_9114.shape)
genotypes_9114 = np.array(dump_data9114[[x for x in mutations_9114]].copy(), dtype=np.float64)
print(genotypes_9114.shape)

(1011,)
(1011, 11)
(2506,)
(2506, 16)


## CV to choose optimal order of interaction

### H1

In [26]:
num_folds = 10
max_order = 4

# set up permutation
np.random.seed(7722)
indices_permuted_6261 = np.random.permutation(np.arange(len(genotypes_6261)))
size_test_6261 = int(1.0/float(num_folds)*len(genotypes_6261))
size_train_6261 = len(genotypes_6261)-size_test_6261
print(size_test_6261, size_train_6261)

# lists to store r squared values
rsq_train_list_6261 = np.zeros((max_order+1, num_folds))
rsq_test_list_6261 = np.zeros((max_order+1, num_folds))


# loop over CV folds
for f in range(num_folds):

    # get train & test sets
    start = int(f*size_test_6261)
    stop = int((f+1)*size_test_6261)
    genos_train_6261 = np.concatenate(
        (genotypes_6261[indices_permuted_6261[:start]], genotypes_6261[indices_permuted_6261[stop:]]))
    genos_test_6261 = genotypes_6261[indices_permuted_6261[start:stop]]
    affinities_train_6261 = np.concatenate(
        (affinities_6261[indices_permuted_6261[:start]], affinities_6261[indices_permuted_6261[stop:]]))
    affinities_test_6261 = affinities_6261[indices_permuted_6261[start:stop]]

    print('Fold: ', f)

    # initialize zero-order (intercept-only) model
    genos_train_6261_previous = np.full(len(genos_train_6261), 1.0)
    genos_test_6261_previous = np.full(len(genos_test_6261), 1.0)

    reg_6261_previous = sm.OLS(affinities_train_6261, genos_train_6261_previous).fit()
    reg_6261_coefs_previous = reg_6261_previous.params

    rsquared_train_6261_previous = reg_6261_previous.rsquared
    rsquared_test_6261_previous = 1-np.sum((affinities_test_6261-reg_6261_previous.predict(
        genos_test_6261_previous))**2)/np.sum((affinities_test_6261-np.mean(affinities_test_6261))**2)
    rsq_train_list_6261[0, f] = rsquared_train_6261_previous
    rsq_test_list_6261[0, f] = rsquared_test_6261_previous

    # mean_pheno_train = np.mean(affinities_train_6261)
    # mean_pheno_test = np.mean(affinities_test_6261)

    # fit models of increasing order
    for order in range(1, max_order+1):
        #print('Order: ',str(order))
        poly_6261_current = PolynomialFeatures(order, interaction_only=True)
        genos_train_6261_current = poly_6261_current.fit_transform(genos_train_6261)
        genos_test_6261_current = poly_6261_current.fit_transform(genos_test_6261)

        reg_6261_current = sm.OLS(affinities_train_6261, genos_train_6261_current).fit()
        reg_6261_coefs_current = reg_6261_current.params
        print(reg_6261_coefs_current) # FIXME
        reg_6261_CIs_current = reg_6261_current.conf_int(alpha=0.05, cols=None)
        reg_6261_stderr = reg_6261_current.bse
        # NOTE: Use mse_resid for BIC if you choose to use that

        rsquared_train_6261_current = reg_6261_current.rsquared
        rsquared_test_6261_current = 1-np.sum((affinities_test_6261-reg_6261_current.predict(
            genos_test_6261_current))**2)/np.sum((affinities_test_6261-np.mean(affinities_test_6261))**2)
        rsq_train_list_6261[order, f] = rsquared_train_6261_current
        rsq_test_list_6261[order, f] = rsquared_test_6261_current
        break # FIXME


# average over folds
mean_rsq_train_6261 = np.mean(rsq_train_list_6261, axis=1)
stdev_rsq_train_6261 = np.std(rsq_train_list_6261, axis=1)
mean_rsq_test_6261 = np.mean(rsq_test_list_6261, axis=1)
stdev_rsq_test_6261 = np.std(rsq_test_list_6261, axis=1)

optimal_6261_order = np.argmax(mean_rsq_test_6261)
print('Optimal order, 6261: ', optimal_6261_order)


101 910
Fold:  0
[-0.40547378  0.83638399  0.75100607 -0.03891322  0.10914697 -0.06055584
 -0.0092875   0.64215407 -0.08378612  0.21335984  0.0926288   0.03920108]
Fold:  1
[-0.42719399  0.83181515  0.74601886 -0.05607121  0.1165019  -0.04872937
 -0.00746616  0.64009972 -0.09017642  0.23220347  0.10987916  0.04005678]
Fold:  2
[-0.43525156  0.84864522  0.74818166 -0.02548082  0.12066663 -0.04879098
 -0.0085349   0.65172879 -0.07411136  0.22896789  0.09048787  0.04710932]
Fold:  3
[-0.42070529  0.820864    0.75140946 -0.02774555  0.12640729 -0.05433158
 -0.01228112  0.63946362 -0.09032306  0.21695345  0.09438272  0.04847423]
Fold:  4
[-0.4454377   0.84998691  0.76820787 -0.05084519  0.12956692 -0.0371712
 -0.01498364  0.65351944 -0.07453505  0.2375457   0.10088164  0.04184763]
Fold:  5
[-0.44705453  0.81925887  0.7557612  -0.02025146  0.13698331 -0.05819001
  0.00288369  0.63680767 -0.08094337  0.24374893  0.0984006   0.04305639]
Fold:  6
[-0.42931184  0.85636818  0.76779195 -0.03161102

In [18]:
# print CV results to file
with open('model_coefs/CR6261_CV_rsquared.csv', 'w') as writefile:
    rsq_writer = csv.writer(writefile)
    rsq_writer.writerow(['Type', 'Order', 'Mean', 'Std'])
    for i in range(len(mean_rsq_train_H1)):
        rsq_writer.writerow(
            ['Train', str(i), mean_rsq_train_H1[i], stdev_rsq_train_H1[i]])
    for i in range(len(mean_rsq_test_H1)):
        rsq_writer.writerow(
            ['Test', str(i), mean_rsq_test_H1[i], stdev_rsq_test_H1[i]])
    writefile.close()


### H9

In [18]:
num_folds = 10
max_order = 5

# set up permutation
np.random.seed(2112)
indices_permuted_H9 = np.random.permutation(np.arange(len(genos_H9)))
size_test_H9 = int(1.0/float(num_folds)*len(genos_H9))
size_train_H9 = len(genos_H9)-size_test_H9
print(size_test_H9, size_train_H9)

# lists to store r squared values
rsq_train_list_H9 = np.zeros((max_order+1, num_folds))
rsq_test_list_H9 = np.zeros((max_order+1, num_folds))


# loop over CV folds
for f in range(num_folds):

    # get train & test sets
    start = int(f*size_test_H9)
    stop = int((f+1)*size_test_H9)
    genos_train_H9 = np.concatenate(
        (genos_H9[indices_permuted_H9[:start]], genos_H9[indices_permuted_H9[stop:]]))
    genos_test_H9 = genos_H9[indices_permuted_H9[start:stop]]
    phenos_train_H9 = np.concatenate(
        (phenos_H9[indices_permuted_H9[:start]], phenos_H9[indices_permuted_H9[stop:]]))
    phenos_test_H9 = phenos_H9[indices_permuted_H9[start:stop]]

    print('Fold: ', f)

    # initialize zero-order (intercept-only) model
    genos_train_H9_previous = np.full(len(genos_train_H9), 1.0)
    genos_test_H9_previous = np.full(len(genos_test_H9), 1.0)

    reg_H9_previous = sm.OLS(phenos_train_H9, genos_train_H9_previous).fit()
    reg_H9_coefs_previous = reg_H9_previous.params

    rsquared_train_H9_previous = reg_H9_previous.rsquared
    rsquared_test_H9_previous = 1-np.sum((phenos_test_H9-reg_H9_previous.predict(
        genos_test_H9_previous))**2)/np.sum((phenos_test_H9-np.mean(phenos_test_H9))**2)
    rsq_train_list_H9[0, f] = rsquared_train_H9_previous
    rsq_test_list_H9[0, f] = rsquared_test_H9_previous

    mean_pheno_train = np.mean(phenos_train_H9)
    mean_pheno_test = np.mean(phenos_test_H9)

    # fit models of increasing order
    for order in range(1, max_order+1):
        #print('Order: ',str(order))
        poly_H9_current = PolynomialFeatures(order, interaction_only=True)
        genos_train_H9_current = poly_H9_current.fit_transform(genos_train_H9)
        genos_test_H9_current = poly_H9_current.fit_transform(genos_test_H9)

        reg_H9_current = sm.OLS(phenos_train_H9, genos_train_H9_current).fit()
        reg_H9_coefs_current = reg_H9_current.params
        reg_H9_CIs_current = reg_H9_current.conf_int(alpha=0.05, cols=None)
        reg_H9_stderr = reg_H9_current.bse

        rsquared_train_H9_current = reg_H9_current.rsquared
        rsquared_test_H9_current = 1-np.sum((phenos_test_H9-reg_H9_current.predict(
            genos_test_H9_current))**2)/np.sum((phenos_test_H9-np.mean(phenos_test_H9))**2)
        rsq_train_list_H9[order, f] = rsquared_train_H9_current
        rsq_test_list_H9[order, f] = rsquared_test_H9_current


# average over folds
mean_rsq_train_H9 = np.mean(rsq_train_list_H9, axis=1)
stdev_rsq_train_H9 = np.std(rsq_train_list_H9, axis=1)
mean_rsq_test_H9 = np.mean(rsq_test_list_H9, axis=1)
stdev_rsq_test_H9 = np.std(rsq_test_list_H9, axis=1)

optimal_H9_order = np.argmax(mean_rsq_test_H9)
print('Optimal order, H9: ', optimal_H9_order)


184 1658
Fold:  0
Fold:  1
Fold:  2
Fold:  3
Fold:  4
Fold:  5
Fold:  6
Fold:  7
Fold:  8
Fold:  9
Optimal order, H9:  4


In [20]:
# print CV results to file
with open('model_coefs/H9_CV_rsquared_'+ep_type+'.csv','w') as writefile:
    rsq_writer = csv.writer(writefile)
    rsq_writer.writerow(['Optimal order: '+str(optimal_H9_order)])
    rsq_writer.writerow(['Type','Order','Mean','Std'])
    for i in range(len(mean_rsq_train_H9)):
        rsq_writer.writerow(['Train',str(i),mean_rsq_train_H9[i],stdev_rsq_train_H9[i]])
    for i in range(len(mean_rsq_test_H9)):
        rsq_writer.writerow(['Test',str(i),mean_rsq_test_H9[i],stdev_rsq_test_H9[i]])
    writefile.close()
    

## Fit final models

### H1

In [26]:
# fit models of increasing order
for order in range(1, optimal_H1_order+1):

    genos_H1_permuted = genos_H1[indices_permuted_H1]
    phenos_H1_permuted = phenos_H1[indices_permuted_H1]
    print('Order: ', str(order))
    poly_H1_current = PolynomialFeatures(order, interaction_only=True)
    genos_H1_current = poly_H1_current.fit_transform(genos_H1_permuted)

    # fit
    reg_H1_current = sm.OLS(phenos_H1_permuted, genos_H1_current).fit()
    reg_H1_coefs_current = reg_H1_current.params
    reg_H1_CIs_current = reg_H1_current.conf_int(
        alpha=0.05/float(len(reg_H1_coefs_current)), cols=None)
    reg_H1_stderr = reg_H1_current.bse
    reg_H1_pvalues = reg_H1_current.pvalues

    num_sig = len(np.where(reg_H1_pvalues < 0.05 /
                  float(len(reg_H1_coefs_current)))[0])
    print(num_sig)

    predicted_phenos_permuted_H1 = reg_H1_current.predict(genos_H1_current)
    rsquared_H1_current = reg_H1_current.rsquared
    print('Params: ', len(reg_H1_coefs_current))
    print('Performance: ', rsquared_H1_current)

    # write model to file
    if order > 0:
        coef_names = poly_H1_current.get_feature_names(
            input_features=mutations)
        with open('model_coefs/H1_'+str(order)+'order_'+ep_type+'.txt', 'w') as writefile:
            coef_writer = csv.writer(writefile, delimiter='\t')
            coef_writer.writerow(['Params: ', len(reg_H1_coefs_current)])
            coef_writer.writerow(['Performance: ', rsquared_H1_current])
            coef_writer.writerow(
                ['Term', 'Coefficient', 'Standard Error', 'p-value', '95% CI lower', '95% CI upper'])
            coef_writer.writerow(['Intercept', reg_H1_coefs_current[0]])
            for i in range(1, len(reg_H1_coefs_current)):
                coef_writer.writerow([','.join(coef_names[i].split(' ')), reg_H1_coefs_current[i], reg_H1_stderr[i],
                                      reg_H1_pvalues[i], reg_H1_CIs_current[i][0], reg_H1_CIs_current[i][1]])
            writefile.close()


Order:  1
10
Params:  12
Performance:  0.7430261110294989
Order:  2
30
Params:  67
Performance:  0.9151429662861195
Order:  3
41
Params:  232
Performance:  0.957979520212946
Order:  4
69
Params:  562
Performance:  0.9827390892301666


### H9

In [27]:
# fit models of increasing order
for order in range(1, optimal_H9_order+1):

    genos_H9_permuted = genos_H9[indices_permuted_H9]
    phenos_H9_permuted = phenos_H9[indices_permuted_H9]
    print('Order: ', str(order))
    poly_H9_current = PolynomialFeatures(order, interaction_only=True)
    genos_H9_current = poly_H9_current.fit_transform(genos_H9_permuted)

    # fit
    reg_H9_current = sm.OLS(phenos_H9_permuted, genos_H9_current).fit()
    reg_H9_coefs_current = reg_H9_current.params
    reg_H9_CIs_current = reg_H9_current.conf_int(
        alpha=0.05/float(len(reg_H9_coefs_current)), cols=None)
    reg_H9_stderr = reg_H9_current.bse
    reg_H9_pvalues = reg_H9_current.pvalues

    num_sig = len(np.where(reg_H9_pvalues < 0.05 /
                  float(len(reg_H9_coefs_current)))[0])
    print(num_sig)

    predicted_phenos_permuted_H9 = reg_H9_current.predict(genos_H9_current)
    rsquared_H9_current = reg_H9_current.rsquared
    print('Params: ', len(reg_H9_coefs_current))
    print('Performance: ', rsquared_H9_current)

    # write model to file
    if order > 0:
        coef_names = poly_H9_current.get_feature_names(
            input_features=mutations)
        with open('model_coefs/H9_'+str(order)+'order_'+ep_type+'.txt', 'w') as writefile:
            coef_writer = csv.writer(writefile, delimiter='\t')
            coef_writer.writerow(['Params: ', len(reg_H9_coefs_current)])
            coef_writer.writerow(['Performance: ', rsquared_H9_current])
            coef_writer.writerow(
                ['Term', 'Coefficient', 'Standard Error', 'p-value', '95% CI lower', '95% CI upper'])
            coef_writer.writerow(['Intercept', reg_H9_coefs_current[0]])
            for i in range(1, len(reg_H9_coefs_current)):
                coef_writer.writerow([','.join(coef_names[i].split(' ')), reg_H9_coefs_current[i], reg_H9_stderr[i],
                                      reg_H9_pvalues[i], reg_H9_CIs_current[i][0], reg_H9_CIs_current[i][1]])
            writefile.close()


Order:  1
11
Params:  12
Performance:  0.7924868580010378
Order:  2
24
Params:  67
Performance:  0.8743026327570844
Order:  3
32
Params:  232
Performance:  0.9292041206369122
Order:  4
41
Params:  562
Performance:  0.9680505884789233
